In [1]:
import tensorflow as tf

import pandas as pd
import numpy as np #데이터 배열화
import os #경로 설정용 
import keras #딥러닝용 패키지
import random #데이터 분산할 때 쓸 랜덤
import cv2 #이미지 읽기용
import math #연산용
import seaborn as sns #matplotlib에 다양한 시각화 기능이 추가된 패키지
from sklearn.metrics import confusion_matrix #분류의 정확성 평가
from sklearn.preprocessing import LabelBinarizer #데이터 전처리용
from sklearn.model_selection import train_test_split #데이터 분할용

import matplotlib.pyplot as plt #데이터 시각화용

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization #사용할 BN, ConV2 등의 계층
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout #사용할 레이어

from tensorflow.keras.applications import VGG16 
from tensorflow.keras.applications.densenet import preprocess_input #tensor나 numpy배열 전처리용

from tensorflow.keras.preprocessing import image #이미지 데이터를 실시간으로 처리하기 위한 도구
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array #이미지 편집을 위한 제너레이터(Affine Transform)

from tensorflow.keras.models import Model #교육 및 추론 기능이 있는 개체로 레이어를 그룹화
from keras.models import Sequential

from tensorflow.keras.optimizers import Adam #Adam 옵티마이저 사용. loss는 categorical_crossentropy 사용

#체크포인트를 두고 저장 + metric이 중지되면 학습률을 감소
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau 

import warnings #경고 무시
warnings.filterwarnings("ignore")

In [2]:
image_size=48

In [8]:
#서버용 그래픽 카드 지정
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
data=[]
labels=[]
random.seed(42)
imagePaths = sorted(list(os.listdir("../jupyter/input/emotion-project-2/train/")))
random.shuffle(imagePaths)
print(imagePaths)

for img in imagePaths:
    path=sorted(list(os.listdir("../jupyter/input/emotion-project-2/train/"+img)))
    for i in path:
        image = cv2.imread("../jupyter/input/emotion-project-2/train/"+img+'/'+i)
        image = cv2.resize(image, (image_size,image_size))
        image = img_to_array(image)
        data.append(image)
        l = label = img
        labels.append(l)

['neutral', 'fear', 'happy', 'sad', 'angry', 'surprise']


In [4]:
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
mlb = LabelBinarizer()
labels = mlb.fit_transform(labels)
print(labels[0])

[0 0 0 1 0 0]


In [5]:
(xtrain,xtest,ytrain,ytest)=train_test_split(data,labels,test_size=0.2,random_state=42)
print(xtrain.shape, xtest.shape)

(50890, 48, 48, 3) (12723, 48, 48, 3)


In [6]:
datagen = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2) #이미지 가공(학습률 향상을 위한 augmentation)
datagen.fit(xtrain)

In [7]:
from tensorflow.keras.optimizers import Adam

model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
x=Flatten()(model.output)
x=Dense(512,activation='relu')(x)
x=Dropout(0.2)(x)
output=Dense(6,activation='softmax')(x) # because we have to predict the AUC
model=Model(model.input,output)

model.compile(
    optimizer = Adam(lr=0.0001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
  )

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

In [8]:
history = model.fit(datagen.flow(xtrain, ytrain, batch_size=32),
               epochs=50,
               validation_data=(xtest, ytest))

Epoch 1/50
1591/1591 [==============================] - 57s 34ms/step - loss: 1.4555 - accuracy: 0.4134 - val_loss: 1.0209 - val_accuracy: 0.6110
Epoch 2/50
1591/1591 [==============================] - 51s 32ms/step - loss: 1.0493 - accuracy: 0.5949 - val_loss: 0.9489 - val_accuracy: 0.6390
Epoch 3/50
1591/1591 [==============================] - 54s 34ms/step - loss: 0.9265 - accuracy: 0.6487 - val_loss: 0.9087 - val_accuracy: 0.6564
Epoch 4/50
1591/1591 [==============================] - 55s 34ms/step - loss: 0.8449 - accuracy: 0.6850 - val_loss: 0.8486 - val_accuracy: 0.6831
Epoch 5/50
1591/1591 [==============================] - 54s 34ms/step - loss: 0.7748 - accuracy: 0.7132 - val_loss: 0.8426 - val_accuracy: 0.6805
Epoch 6/50
1591/1591 [==============================] - 56s 35ms/step - loss: 0.7072 - accuracy: 0.7403 - val_loss: 0.7871 - val_accuracy: 0.7148
Epoch 7/50
1591/1591 [==============================] - 57s 36ms/step - loss: 0.6303 - accuracy: 0.7722 - val_loss: 0.7990 -

In [9]:
model.evaluate(xtest, ytest, batch_size = 32)

398/398 [==============================] - 4s 9ms/step - loss: 0.5817 - accuracy: 0.8867


[0.581672191619873, 0.8867405652999878]

In [10]:
model.evaluate(xtrain, ytrain, batch_size = 32)

1591/1591 [==============================] - 15s 9ms/step - loss: 0.0512 - accuracy: 0.9820


[0.05116046592593193, 0.9819610714912415]

In [12]:
model.save('adam_0429_float32_48.h5')